In [2]:
## use RNA-STAR conda environment
from pathlib import Path
import traceback
import pandas as pd
import numpy as np
import re
from scipy.stats import fisher_exact

### Functions

In [47]:
def create_mask(df, colnames):
    """
    NOTES:
    * Select columns that contain "Deletions" and put them in a list
    * Use set() to remove duplicates, since sets can only contain unique vals
    * Pass column names in list to dataframe to create a mask that drops rows
    where Deletions == 0 and there are nulls
    """
    del_list = list(set([col for col in colnames if re.search(r"Deletions", col)]))
    mask = ~(df[del_list] == 0).any(axis = 1) & (df.notna().all(axis = 1))
    return mask

def calc_pval(df_merged, merged_colnames, rep_list, subfolder):
    for rep in rep_list: 
        wt_7ko = subfolder.split("-")[0]
        cyto_nuc = subfolder.split("-")[1]
        bs_del_col = [col for col in merged_colnames 
                        if re.search(f"{rep}_Deletions_BS", col)]
        nbs_del_col = [col for col in merged_colnames 
                        if re.search(f"{rep}_Deletions_NBS", col)]

        ## Group corresponding BS/NBS into separate lists (not modifying original df)
        bs_base_pattern = re.compile(fr"{rep}_(A|C|G|T)_BS$")
        nbs_base_pattern = re.compile(fr"{rep}_(A|C|G|T)_NBS$")
        pattern_dict = {f"{rep}_Bases_BS": [col for col in merged_colnames 
                                            if bs_base_pattern.match(col)],
                        f"{rep}_Bases_NBS": [col for col in merged_colnames 
                                                if nbs_base_pattern.match(col)]}

        ## Define names of base and deletion BS/NBS columns
        base_cols = [f"{wt_7ko}_{cyto_nuc}_{rep}_TotalBases_BS", 
                        f"{wt_7ko}_{cyto_nuc}_{rep}_TotalBases_NBS"]
        del_cols = [bs_del_col[0], nbs_del_col[0]]
        
        ## Define generic entries for 2x2 contingency table
        fisher_cols = [base_cols[0], 
                        del_cols[0], 
                        base_cols[1], 
                        del_cols[1]]

        ## Calculate p-values
        for col, key in zip(base_cols, pattern_dict):
            if col not in df_merged.columns:
                df_merged[col] = df_merged[pattern_dict[key]].sum(axis = 1)

        if set(fisher_cols).issubset(df_merged.columns):
            df_merged = df_merged.dropna(subset = fisher_cols)
            arr = df_merged[fisher_cols].values.reshape(-1, 2, 2) 
            pvals = [fisher_exact(table)[1] for table in arr]
            df_merged[f"{rep}_Pvalue"] = pvals

    return df_merged

### Loading in data

In [11]:
current_path = Path.cwd()
input_dir = current_path/"merged"
subfolder = "7KO-Cyto"
processed_folder = current_path/"pvals"/subfolder

In [21]:
tsv_list = sorted(input_dir.glob("*.tsv")) 
df_list = [pd.read_csv(str(file), sep = "\t") for file in tsv_list]

### Dataframe merging

In [31]:
df1_colnames = df_list[0].columns.tolist()
selected_colnames = df1_colnames[0:17]
init_mask = create_mask(df_list[0], df1_colnames)
df = df_list[0].loc[init_mask]

merged = pd.merge(df, df_list[1], on = selected_colnames, how = "outer")

In [ ]:
df_merged = merged.dropna()
merged_colnames = df_merged.columns.tolist()
rep_list = sorted(
    set([re.search(r"(Rep\d+)", col).group(1) for col in merged_colnames 
        if re.search(r"(Rep\d+)", col)]), 
        key = lambda x: int(re.search(r"Rep(\d+)", x).group(1))
)

rep_list

['Rep1', 'Rep2', 'Rep3']

### Calculating p-values (DEBUG)

In [48]:
calc_pval(df_merged, merged_colnames, rep_list, subfolder)

,TranscriptID,Motif,Region,Chrom,Strand,TranscriptPosStart,TranscriptPosEnd,TranscriptModBase,GenomicModBase,TranscriptLength,...,7KO_StdDeletionRate_NBS,7KO_Cyto_Rep1_TotalBases_BS,7KO_Cyto_Rep1_TotalBases_NBS,Rep1_Pvalue,7KO_Cyto_Rep2_TotalBases_BS,7KO_Cyto_Rep2_TotalBases_NBS,Rep2_Pvalue,7KO_Cyto_Rep3_TotalBases_BS,7KO_Cyto_Rep3_TotalBases_NBS,Rep3_Pvalue


In [ ]:
## Clearly there's something going wrong here!